## Testa MESA

In [3]:
import mesa 
import mesa.time
from mesa import Model, Agent
from mesa import RandomActivation

class TestAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        print(f"Agent {self.unique_id} kör sitt step.")

class TestModel(Model):
    def __init__(self, N):
        self.schedule = RandomActivation(self)
        for i in range(N):
            a = TestAgent(i, self)
            self.schedule.add(a)

    def step(self):
        self.schedule.step()

# Kör modellen
model = TestModel(N=3)
for i in range(5):
    print(f"\n--- Step {i} ---")
    model.step()


ModuleNotFoundError: No module named 'mesa.time'